**Create Qiskit QuantumCircuit from Braket Circuit objects!**

The implementation should be able to handle the Braket circuit containing the following appropriately:
Supported gates
Supported result types and observables
Supported noise instructions
Verbatim blocks
The implementation should be accompanied by unit tests with full coverage.

In [ ]:
import qiskit
from qiskit import QuantumCircuit
from qiskit.circuit.library import reset, initialize
from braket.circuits import Circuit as BraketCircuit
from braket.circuits import instr
from braket.circuits.result_types import ResultType

In [ ]:
def from_braket_circuit(circuit: Circuit) -> QuantumCircuit:
    # Create an empty Qiskit circuit
    qiskit_circuit = QuantumCircuit(Circuit.qubit_count)

    # Convert Braket gates to Qiskit gates
    for braket_instruction in Circuit.instructions:
        if isinstance(braket_instruction, instr.Gate):
            qiskit_circuit.append(convert_braket_gate(braket_instruction), braket_instruction.targets)
        elif isinstance(braket_instruction, instr.Verb):
            qiskit_circuit.append(convert_braket_verbatim(braket_instruction), braket_instruction.targets)
            
     # Add measurements and result types
    for result_type in Circuit.result_types:
        if result_type == ResultType.Probability:
            qiskit_circuit.measure_all()
        elif result_type == ResultType.Amplitude:
            qiskit_circuit.barrier()
        elif result_type == ResultType.Expectation:
            observable = braket_circuit.get_observable_instructions()[0]
            qiskit_circuit.append(convert_braket_observable(observable), observable.targets)

    return qiskit_circuit

In [ ]:
def convert_braket_gate(braket_gate):
    if braket_gate.name == 'X':
        return qiskit.circuit.library.XGate()
    elif braket_gate.name == 'Y':
        return qiskit.circuit.library.YGate()
    elif braket_gate.name == 'Z':
        return qiskit.circuit.library.ZGate()
    elif braket_gate.name == 'H':
        return qiskit.circuit.library.HGate()
    elif braket_gate.name == 'CNOT':
        return qiskit.circuit.library.CXGate()

In [ ]:
def convert_braket_observable(braket_observable):
    if braket_observable.name == 'PauliX':
        return qiskit.circuit.library.PauliX()
    elif braket_observable.name == 'PauliY':
        return qiskit.circuit.library.PauliY()
    elif braket_observable.name == 'PauliZ':
        return qiskit.circuit.library.PauliZ()


def convert_braket_verbatim(braket_verbatim):
    if braket_verbatim.name == 'RESET':
        return reset.Reset()
    elif braket_verbatim.name == 'INITIALIZE':
        return initialize.Initialize(braket_verbatim.params[0].tolist())

In [ ]:
# Unit Tests
def test_braket_to_qiskit():
    # Create a Braket circuit object
    braket_circuit = BraketCircuit().h(0).cx(0, 1).h(1).x(0).z(1).m([0, 1])

    # Convert Braket circuit to Qiskit circuit
    qiskit_circuit = braket_to_qiskit(braket_circuit)

    # Verify the gates and measurements in Qiskit circuit
    assert isinstance(qiskit_circuit[0], qiskit.circuit.library.HGate)
    assert isinstance(qiskit_circuit[1], qiskit.circuit.library.CXGate)
    assert isinstance(qiskit_circuit[2], qiskit.circuit.library.HGate)
    assert isinstance(qiskit_circuit[3], qiskit.circuit.library.XGate)
    assert isinstance(qiskit_circuit[4], qiskit.circuit.library.ZGate)
    assert qiskit_circuit.num_qubits == 2
    assert qiskit_circuit.num_clbits == 2
    assert qiskit_circuit.count_ops() == {'h': 2, 'cx': 1, 'x': 1, 'z': 1, 'measure': 2}

    print("Braket to Qiskit conversion successful.")


# Run unit tests
test_braket_to_qiskit()
